<a href="https://colab.research.google.com/github/bismark-joe/ML-project_experience/blob/master/Dataset_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Normalize, Preprocess and Prepare for Model
This is the phase where we:

*   load in our data from Google Drive,

*   the data is stored in an Excel file with about 15 Sheets in it and 6 consistent columns ('Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'), in each Sheet,

*   merge the all the data together into one and save it as csv file instead of excel,

*   extract the 'Content' column and clean,

*   convert the values in other columns to 0 where the value is null otherwise save it as 1,

*   List item
*   List item



#Importing the Dependencies/Libraries

In [ ]:
import html
import re
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import drive

#Connecting to Google Drive

In [ ]:
drive.mount('/content/drive')

#connecting to the specific folder 'Banji' where Data is saved on G.Drive.
BASE_DIR = "/content/drive/MyDrive/Banji"

#Save the Folder as the Base directory where every other file stores
os.chdir(BASE_DIR)

Mounted at /content/drive


Load the data 'Banji_dataset.xlsx' from Drive

In [ ]:
# Load the Excel file
file_path = 'Banji_dataset.xlsx'
xls = pd.ExcelFile(file_path)

# Iterate through each sheet and print the column names
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)

    # Clean up the column names (remove extra spaces)
    df.columns = df.columns.str.strip()

    print(f"Sheet Name: {sheet_name}")
    print("Column Names:")
    print(df.columns)
    print()

Sheet Name: #afonja
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #elrufai
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #Endsars
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #herdsmen
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #igboho
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')



/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Sheet Name: #ipob
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #laimuhamed
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #lekki
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #mbuharior #BUHARI
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: #biafra
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: HATE SPEECH
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: RELIGIOUS HATE SPEECH
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other'], dtype='object')

Sheet Name: ETHNIC HATE SPEECH
Column Names:
Index(['Content', 'Hate', 'Non-Hate', 'Religious',

Merge the Excel Sheets in the data together

In [ ]:
# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Define the required columns
required_columns = ['Content', 'Hate', 'Non-Hate', 'Religious', 'Ethnicity', 'Other']

# Initialize a list to store the names of sheets with problems
sheets_with_problems = []

# Iterate through each sheet and concatenate the data to the merged DataFrame
for sheet_name in xls.sheet_names:
    try:
        df = pd.read_excel(xls, sheet_name)

        # Clean up the column names (remove extra spaces)
        df.columns = df.columns.str.strip()

        # Filter and keep only the required columns
        df = df[required_columns]

        merged_df = pd.concat([merged_df, df], ignore_index=True)

        print(f'Sheet "{sheet_name}" merged successfully.')
    except KeyError:
        sheets_with_problems.append(sheet_name)
        print(f'Sheet "{sheet_name}" has problems and was skipped.')

# Save the merged DataFrame as a CSV file
output_csv = 'normalized_dataset.csv'
merged_df.to_csv(output_csv, index=False)

if sheets_with_problems:
    print(f'Merged dataset saved as {output_csv} with some sheets skipped due to problems: {sheets_with_problems}')
else:
    print(f'Merged dataset saved as {output_csv}')


Sheet "#afonja" merged successfully.
Sheet "#elrufai" merged successfully.
Sheet "#Endsars" merged successfully.
Sheet "#herdsmen" merged successfully.
Sheet "#igboho" merged successfully.


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Sheet "#ipob" merged successfully.
Sheet "#laimuhamed" merged successfully.
Sheet "#lekki" merged successfully.
Sheet "#mbuharior #BUHARI" merged successfully.
Sheet "#biafra" merged successfully.
Sheet "HATE SPEECH" merged successfully.
Sheet "RELIGIOUS HATE SPEECH" merged successfully.
Sheet "ETHNIC HATE SPEECH" merged successfully.
Sheet "OTHERS" merged successfully.
Sheet "non hate speech" merged successfully.
Merged dataset saved as normalized_dataset.csv


#Data Cleaning and Preprocessing
This updated code performs the cleaning operations on the 'Content' column and then saves the cleaned DataFrame as 'cleaned_normalized_dataset.csv'.

In [ ]:
# Load the dataset
new_sample = pd.read_csv("normalized_dataset.csv")

# Clean the 'Content' column
tweets = new_sample["Content"].copy()  # Make a copy to avoid modifying the original DataFrame

for i in range(len(tweets)):
    if isinstance(tweets[i], str):
        # Decode HTML entities
        tweets[i] = html.unescape(tweets[i])

        # Remove special characters, mentions, hashtags, URLs, and extra spaces
        tweets[i] = re.sub(r"(@[A-Za-z0-9_]+)|[^\w\s]|#|http\S+", "", tweets[i])
        tweets[i] = re.sub(r"\s{2,}", " ", tweets[i])

        # Convert text to lowercase
        tweets[i] = tweets[i].lower()

# Update the 'Content' column in the DataFrame
new_sample["Content"] = tweets

# Define the columns for features
features = new_sample[["Hate", "Non-Hate", "Religious", "Ethnicity", "Other"]]

# Define a function to convert various forms of "Yes" to 1 and empty cells to 0
def convert_value(value):
    if pd.isna(value) or value == "":
        return 0
    return 1

# Apply the function to the DataFrame using .loc indexer
for column in features:
    new_sample.loc[:, column] = features[column].apply(convert_value)

# Save the cleaned dataset as a new CSV file
cleaned_csv = "cleaned_normalized_dataset.csv"
new_sample.to_csv(cleaned_csv, index=False)

print(f'Cleaned dataset saved as {cleaned_csv}')

<ipython-input-5-a223a8162a82>:33: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_sample.loc[:, column] = features[column].apply(convert_value)


Cleaned dataset saved as cleaned_normalized_dataset.csv


## Preparing Dataset for Each Stage of SVM Models



In [ ]:
# Load the cleaned dataset
df = pd.read_csv('cleaned_normalized_dataset.csv')

*   Turn "Hate" column value where we have 0 to 1, if any of the hate-types have 1
*   Remove rows with all zeros

In [ ]:
# Update rows where Hate is 0 and any of the hate-type columns is 1
hate_types = ['Religious', 'Ethnicity', 'Other']

for hate_type in hate_types:
    df.loc[(df['Hate'] == 0) & (df[hate_type] == 1), 'Hate'] = 1

For the Frist Stage

In [ ]:
# Remove rows where all values are 0 or where the "Content" column is empty
df = df[~((df.drop('Content', axis=1) == 0).all(axis=1) | df['Content'].isna())]


# Balancing the dataset
# Number of rows to keep
num_rows_to_keep = 3781

# Filter rows where Non-Hate is 1
non_hate_rows = df[df['Non-Hate'] == 1]

# Randomly sample a subset of rows to keep
rows_to_keep = non_hate_rows.sample(n=num_rows_to_keep, random_state=42)

# Combine the subset of rows to keep with the other rows where Non-Hate is 0
final_df = pd.concat([df[df['Non-Hate'] == 0], rows_to_keep])
df = final_df
df = df.reset_index(drop=True)  # Reset index and drop the old index column


# Save the dataset
cleaned_csv = "cleaned_dataset_without_missing_text.csv"
df.to_csv(cleaned_csv, index=False)

In [ ]:
df.head()
df['Hate'].value_counts()

1    11188
0     3779
Name: Hate, dtype: int64

For the Second Stage

In [ ]:
# Note: dataset_without_non_hate_svm_2 == df2

# Remove rows where the value in the "Non-Hate" column is 1 For SVM 2
df2 = df[df['Non-Hate'] != 1]
df2 = pd.DataFrame(df2)
df2_save = df2

# Merge columns and create a new column with the summation
df2['Religion_Ethnicity'] = df2['Religious'] + df2['Ethnicity']

# Reset index to re-number rows
df2 = df2.reset_index(drop=True)  # Reset index and drop the old index column
df2 = df2.drop(columns=['Hate', 'Non-Hate', 'Religious','Ethnicity'])


# Save the cleaned dataset as a new CSV file
cleaned_csv = "dataset_without_non_hate_svm_2.csv"
df2.to_csv(cleaned_csv, index=False)

In [ ]:
df2

,Content,Other,Religion_Ethnicity
0,such senseless act could only make sense to f...,1,0
1,the nigerian govt will not go after them but r...,1,0
2,ipob remains the only constant obstacle on the...,0,1
3,esn was created in response to the shameful fa...,0,1
4,ohamadike 1 who say men no dey taaaa ndi ara ...,1,0
...,...,...,...
11181,stop associating ipob radiobiafra with compro...,1,0
11182,stop associating ipob radiobiafra with compro...,1,0
11183,dont be deceived by the zoo biafra is for real...,1,0
11184,free biafra from nigeoria now,1,0


For the Third Stage

In [ ]:
# Note dataset_without_others_and_non_hate_svm_3 == df3

# For SVM 3, Remove all rows where other hate is 1
# Remove rows where the value in the "Other" column is 1 For SVM 3

df3 = df2_save[df2_save['Other'] != 1]

# Reset index to re-number rows
df3  = df3 .reset_index(drop=True)  # Reset index and drop the old index column
df3 = df3.drop(columns=['Hate', 'Non-Hate', 'Other', 'Religion_Ethnicity'])

df31 = df3[df3['Religious'] == df3['Ethnicity']]
rows_to_remove = df3[df3.isin(df31.to_dict(orient='list')).all(axis=1)].index


df3 = df3.drop(rows_to_remove)
# Save the cleaned dataset as a new CSV file
cleaned_csv = "dataset_without_others_and_non_hate_svm_3.csv"
df3.to_csv(cleaned_csv, index=False)

In [ ]:
df3.shape

(939, 3)

### Codes with Modification

Preparing Dataset for each stage of SVM Models

In [ ]:
# Load the cleaned dataset
df = pd.read_csv('cleaned_normalized_dataset.csv')

In [ ]:
df.head()

,Content,Hate,Non-Hate,Religious,Ethnicity,Other
0,such senseless act could only make sense to f...,1,0,0,0,1
1,herdsmen education will curb insecurity â gumi...,0,1,0,0,0
2,and will not enforce the anti open grazing la...,0,1,0,0,0
3,gumi charges fg to build schools for bandits t...,0,1,0,0,0
4,gumi charges government to build schools for b...,0,1,0,0,0


from matplotlib import pyplot as plt
_df_0['Hate'].plot(kind='hist', bins=20, title='Hate')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Non-Hate'].plot(kind='hist', bins=20, title='Non-Hate')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Other'].plot(kind='hist', bins=20, title='Other')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Content').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_11['Non-Hate'].plot(kind='line', figsize=(8, 4), title='Non-Hate')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['Other'].plot(kind='line', figsize=(8, 4), title='Other')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='Hate', y='Content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Non-Hate', y='Content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Other', y='Content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

*   Turn "Hate" column value where we have 0 to 1, if any of the hate-types have 1
*   Remove rows with all zeros

In [ ]:
# Update rows where Hate is 0 and any of the hate-type columns is 1
hate_types = ['Religious', 'Ethnicity', 'Other']

for hate_type in hate_types:
    df.loc[(df['Hate'] == 0) & (df[hate_type] == 1), 'Hate'] = 1

The First stage + Balancing the Dataset

In [ ]:
# Remove rows where all values are 0 or where the "Content" column is empty
df = df[~((df.drop('Content', axis=1) == 0).all(axis=1) | df['Content'].isna())]


# Number of rows to keep
num_rows_to_keep = 11243

# Filter rows where Non-Hate is 1
non_hate_rows = df[df['Non-Hate'] == 1]

# Randomly sample a subset of rows to keep
rows_to_keep = non_hate_rows.sample(n=num_rows_to_keep, random_state=42)

# Combine the subset of rows to keep with the other rows where Non-Hate is 0
final_df = pd.concat([df[df['Non-Hate'] == 0], rows_to_keep])
df = final_df
df = df.reset_index(drop=True)  # Reset index and drop the old index column

cleaned_csv = "cleaned_dataset_without_missing_text.csv"
df.to_csv(cleaned_csv, index=False)

df.head()
#df['Hate'].value_counts()
#df['Non-Hate'].value_counts()

,Content,Hate,Non-Hate,Religious,Ethnicity,Other
0,such senseless act could only make sense to f...,1,0,0,0,1
1,the nigerian govt will not go after them but r...,1,0,0,0,1
2,ipob remains the only constant obstacle on the...,1,0,0,1,0
3,esn was created in response to the shameful fa...,1,0,0,1,0
4,ohamadike 1 who say men no dey taaaa ndi ara ...,1,0,0,0,1


from matplotlib import pyplot as plt
_df_16['Ethnicity'].plot(kind='hist', bins=20, title='Ethnicity')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_17['Other'].plot(kind='hist', bins=20, title='Other')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_18.groupby('Content').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_19.plot(kind='scatter', x='Ethnicity', y='Other', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Non-Hate']
  ys = series['Ethnicity']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_23.sort_values('Non-Hate', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Non-Hate')
_ = plt.ylabel('Ethnicity')

from matplotlib import pyplot as plt
_df_24['Ethnicity'].plot(kind='line', figsize=(8, 4), title='Ethnicity')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_25['Other'].plot(kind='line', figsize=(8, 4), title='Other')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['Content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='Ethnicity', y='Content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_27['Content'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_27, x='Other', y='Content', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

For the Second Stage

In [ ]:
# Note: dataset_without_non_hate_svm_2 == df2

# Remove rows where the value in the "Non-Hate" column is 1 For SVM 2
df2 = df[df['Non-Hate'] != 1]
# Reset index to re-number rows
df2 = df2.reset_index(drop=True)  # Reset index and drop the old index column
df2 = df2.drop(columns=['Hate', 'Non-Hate'])



# The number of 'other' is about 10189 and the rest (Ethnicity or Religion) is 997 so this dataset has to be balanced also

# Number of rows to keep
num_rows_to_keep = 997

# Filter rows where Non-Hate is 1
non_hate_rows = df2[df2['Other'] == 1]

# Randomly sample a subset of rows to keep
rows_to_keep = non_hate_rows.sample(n=num_rows_to_keep, random_state=42)

# Combine the subset of rows to keep with the other rows where Non-Hate is 0
final_df = pd.concat([df2[df2['Other'] == 0], rows_to_keep])

df2= final_df
df2 = df2.reset_index(drop=True)  # Reset index and drop the old index column
# Merge columns and create a new column with the summation
df2['Religion_Ethnicity'] = df2['Religious'] + df2['Ethnicity']

# Save the cleaned dataset as a new CSV file
cleaned_csv = "dataset_without_non_hate_svm_2.csv"
df2.to_csv(cleaned_csv, index=False)

df2.head()

#df2['Other'].value_counts()

,Content,Religious,Ethnicity,Other,Religion_Ethnicity
0,ipob remains the only constant obstacle on the...,0,1,0,1
1,esn was created in response to the shameful fa...,0,1,0,1
2,igbo people are the most tribalistic and corru...,0,1,0,1
3,the fulani value the lives of their cows far ...,0,1,0,1
4,bokoharam terrorism terrorists bandits ipob es...,0,1,0,1


from matplotlib import pyplot as plt
import seaborn as sns
_df_28.groupby('Content').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Religious']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Religious'}, axis=1)
              .sort_values('Religious', ascending=True))
  xs = counted['Religious']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_29.sort_values('Religious', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Religious')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Ethnicity']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Ethnicity'}, axis=1)
              .sort_values('Ethnicity', ascending=True))
  xs = counted['Ethnicity']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_30.sort_values('Ethnicity', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Ethnicity')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Other']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Other'}, axis=1)
              .sort_values('Other', ascending=True))
  xs = counted['Other']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_31.sort_values('Other', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Other')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Religion_Ethnicity']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Religion_Ethnicity'}, axis=1)
              .sort_values('Religion_Ethnicity', ascending=True))
  xs = counted['Religion_Ethnicity']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_32.sort_values('Religion_Ethnicity', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Religion_Ethnicity')
_ = plt.ylabel('count()')

For the Third Stage

In [ ]:
# Note dataset_without_others_and_non_hate_svm_3 == df3

# For SVM 3, Remove all rows where other hate is 1
# Remove rows where the value in the "Other" column is 1 For SVM 3

df3 = df2[df2['Other'] != 1]

# Reset index to re-number rows
df3  = df3 .reset_index(drop=True)  # Reset index and drop the old index column
df3 = df3.drop(columns=['Other'])

df31 = df3[df3['Religious'] == df3['Ethnicity']]

# Get the indices of rows in df_full that are also present in df_subset
rows_to_remove = df3[df3.isin(df31.to_dict(orient='list')).all(axis=1)].index

# Remove the rows from df_full using the obtained indices
df3 = df3.drop(rows_to_remove).reset_index(drop=True)



# The count of Religion (265) and Ethnicity (674) also varies, so I balanced this also
# Number of rows to keep
num_rows_to_keep = 265

# Filter rows where Non-Hate is 1
non_hate_rows = df3[df3['Ethnicity'] == 1]

# Randomly sample a subset of rows to keep
rows_to_keep = non_hate_rows.sample(n=num_rows_to_keep, random_state=42)

# Combine the subset of rows to keep with the other rows where Non-Hate is 0
final_df = pd.concat([df3[df3['Ethnicity'] == 0], rows_to_keep])

df3= final_df
df3 = df3.reset_index(drop=True)  # Reset index and drop the old index column
df3 = df3.drop(columns=['Religion_Ethnicity'])

# Save the cleaned dataset as a new CSV file
cleaned_csv = "dataset_without_others_and_non_hate_svm_3.csv"
df3.to_csv(cleaned_csv, index=False)

df3.head()

#df3['Religious'].value_counts()
#df3['Ethnicity'].value_counts()

,Content,Religious,Ethnicity
0,madness of the highest order if gumi or whate...,1,0
1,delete threats esn we curse workers of iniquit...,1,0
2,coranavirus not a single pastor in africa prop...,1,0
3,i dunno when youll have common sense enough t...,1,0
4,to all my friends and families and enemies who...,1,0


from matplotlib import pyplot as plt
import seaborn as sns
_df_33.groupby('Content').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Religious']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Religious'}, axis=1)
              .sort_values('Religious', ascending=True))
  xs = counted['Religious']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_34.sort_values('Religious', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Religious')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Ethnicity']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Ethnicity'}, axis=1)
              .sort_values('Ethnicity', ascending=True))
  xs = counted['Ethnicity']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_35.sort_values('Ethnicity', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Content')):
  _plot_series(series, series_name, i)
  fig.legend(title='Content', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Ethnicity')
_ = plt.ylabel('count()')